<a href="https://colab.research.google.com/github/ravi-raj-97/big_data_pharmacy_county/blob/ashish/CS6350_BigData_Project_SubProb_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Sub-problem 5:*** Correlation between the general climate of a county with the health of its population and availability of pharmacies.

Get Files

In [1]:
!wget -nc https://raw.githubusercontent.com/ravi-raj-97/big_data_pharmacy_county/main/Pharmacy-County.xlsx
!wget -nc https://raw.githubusercontent.com/ravi-raj-97/big_data_pharmacy_county/main/Demography_USA.xlsx

--2022-04-24 19:10:02--  https://raw.githubusercontent.com/ravi-raj-97/big_data_pharmacy_county/main/Pharmacy-County.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2173314 (2.1M) [application/octet-stream]
Saving to: ‘Pharmacy-County.xlsx’

Pharmacy-County.xls 100%[===================>]   2.07M  --.-KB/s    in 0.06s   

2022-04-24 19:10:03 (32.1 MB/s) - ‘Pharmacy-County.xlsx’ saved [2173314/2173314]

--2022-04-24 19:10:03--  https://raw.githubusercontent.com/ravi-raj-97/big_data_pharmacy_county/main/Demography_USA.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request

Form dataframes

In [2]:
import pandas as pd

In [3]:
pharmacy_county=pd.ExcelFile("Pharmacy-County.xlsx")
demography_usa=pd.ExcelFile("Demography_USA.xlsx")

In [4]:
pharmacy_county.sheet_names[0:5]

['Alaska', 'Arkansas', 'California', 'Colarado', 'Connecticut']

In [5]:
demography_usa.sheet_names[:6]

['Demography USA', 'Population', 'Housing', 'Households', 'Ages', 'Genders']

In [6]:
pharmacy_county_statewise={}

for state in pharmacy_county.sheet_names:
  pharmacy_county_statewise[state]=pd.io.excel.ExcelFile.parse(pharmacy_county, state).drop(0)

In [7]:
pharmacy_county_statewise.keys()

dict_keys(['Alaska', 'Arkansas', 'California', 'Colarado', 'Connecticut', 'Delaware', 'Flordia', 'Hawaii', 'Idaho', 'Kansas', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Jersey', 'North Carolina', 'North Dakota', 'Oklahoma', 'Oregon', 'Rhode island', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Wisconsin', 'Other State'])

In [8]:
demography_usa_categories={}

demography_usa_categories['Climate']=pd.io.excel.ExcelFile.parse(demography_usa, 'Demography USA')
demography_usa_categories['Population']=pd.io.excel.ExcelFile.parse(demography_usa, 'Population')
demography_usa_categories['Diseases']=pd.io.excel.ExcelFile.parse(demography_usa, 'Diseases')

In [9]:
demography_usa_categories.keys()

dict_keys(['Climate', 'Population', 'Diseases'])



---

## 1. Correlation between the general climate of a county with the health of its population

In [44]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

Create joined dataframe and splitting into test and train sets

In [31]:
print(demography_usa_categories['Diseases'].shape)
print(demography_usa_categories['Climate'].shape)

(3219, 16)
(3219, 12)


In [24]:
demography_disease_climate=demography_usa_categories['Diseases'].merge(demography_usa_categories['Climate'],on=['OBJECTID'],how='inner',suffixes=('', '_y'))
demography_disease_climate.drop(demography_disease_climate.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

In [30]:
demography_disease_climate.shape

(3219, 23)

In [38]:
demography_disease_climate['IECC_Zone']=demography_disease_climate['IECC Climate Zone'].astype(str)+demography_disease_climate['IECC Moisture Regime']

In [39]:
demography_disease_climate.shape

(3219, 24)

In [40]:
demography_disease_climate_train,demography_disease_climate_test=train_test_split(demography_disease_climate,train_size=0.7)

In [41]:
print(demography_disease_climate_train.shape)
print(demography_disease_climate_test.shape)

(2253, 24)
(966, 24)


Preprocess

In [52]:
feature_columns=['Prevalence of obesity', 'Hypertension', 'Diabetes', 'CVD', 'HIV/AIDS', 'cvd_100k', 'hypertension_100k']
class_columns=['IECC_Zone']#,'IECC Climate Zone', 'IECC Moisture Regime', 'BA Climate Zone', 'Temp']

In [57]:
x_train=demography_disease_climate_train[feature_columns].values
x_test=demography_disease_climate_test[feature_columns].values

y_train=demography_disease_climate_train[class_columns].apply(lambda col: pd.factorize(col, sort=True)[0]).values
y_test=demography_disease_climate_test[class_columns].apply(lambda col: pd.factorize(col, sort=True)[0]).values

In [58]:
y_train

array([[ 6],
       [ 9],
       [ 1],
       ...,
       [ 6],
       [11],
       [ 6]])

Using SVM

In [91]:
result_string=[]
for k in range(1,len(feature_columns)+1):
  svm = make_pipeline(StandardScaler(), PCA(n_components=k), SVC(gamma='auto'),)
  _=svm.fit(x_train,y_train)
  score=svm.score(x_test,y_test)
  result_string.append(f"num_features = {k} : {score}")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [92]:
for result in result_string:
  print(result)

num_features = 1 : 0.2329192546583851
num_features = 2 : 0.2587991718426501
num_features = 3 : 0.3250517598343685
num_features = 4 : 0.37267080745341613
num_features = 5 : 0.36438923395445133
num_features = 6 : 0.35714285714285715
num_features = 7 : 0.34886128364389235


Decision Trees

In [93]:
from sklearn.tree import DecisionTreeClassifier

In [113]:
svm = make_pipeline(StandardScaler(), DecisionTreeClassifier(max_features=4))
_=svm.fit(x_train,y_train)
score=svm.score(x_test,y_test)

In [114]:
print(score)

0.30745341614906835


In [120]:
max_score=0
depth=1
for k in range(1,100):
  tree = make_pipeline(StandardScaler(), DecisionTreeClassifier(max_depth=k))
  _=tree.fit(x_train,y_train)
  score=tree.score(x_test,y_test)
  if score>max_score:
    max_score=score
    depth=k
print(f"depth = {depth} : score = {max_score}")

depth = 7 : score = 0.3581780538302277


Random Forest

In [121]:
from sklearn.ensemble import RandomForestClassifier

In [127]:
max_score=0
depth=1
for k in range(1,len(feature_columns)+1):
  tree = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=k))
  _=tree.fit(x_train,y_train.ravel())
  score=tree.score(x_test,y_test.ravel())
  if score>max_score:
    max_score=score
    depth=k
print(f"depth = {depth} : score = {max_score}")

depth = 7 : score = 0.38716356107660455


Multi Layer Perceptron

In [128]:
from sklearn.neural_network import MLPClassifier

In [134]:
max_score=0
num_features=1
for k in range(1,len(feature_columns)+1):
  nn = make_pipeline(StandardScaler(), PCA(n_components=k), MLPClassifier(random_state=1, max_iter=10000))
  _=nn.fit(x_train,y_train.ravel())
  score=nn.score(x_test,y_test.ravel())
  if score>max_score:
    max_score=score
    num_features=k
print(f"num_features = {num_features} : score = {max_score}")

num_features = 5 : score = 0.41200828157349895


Conclusion:

*Using various classifiers, a maximum test score of 41.2% could be achieved which indicates that there is weak or no correlation between thee health of the county and the climate*

---

## 2. Correlation between Health of the county and number of pharmacies